<a href="https://colab.research.google.com/github/shiomasa1218/Colaboratory/blob/master/CNNshio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
# drive mean root directory of  google drive
!ls ./gdrive/'My Drive'/'Kumamoto-Univ'/'Graduationwork'/'exefolder'/'test_folder_name'

carpet1_2.0_afterlinear   sponge-y_2.0_afterlinear
carpet2_2.0_afterlinear   stonetile1_2.0_afterlinear
carpet3_2.0_afterlinear   whiteitile1_2.0_afterlinear
sponge-g_2.0_afterlinear  woodtile1_2.0_afterlinear


In [3]:
# This only needs to be done once per notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# check auth
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

    100% |████████████████████████████████| 993kB 22.1MB/s 


In [4]:
cd gdrive/'My Drive'/'Kumamoto-Univ'/'Graduationwork'/'exefolder'/

/content/gdrive/My Drive/Kumamoto-Univ/Graduationwork/exefolder


In [12]:
import tensorflow as tf
import os
import random
import numpy as np
import csv
import pandas as pd
import input_data
import time
import datetime
from sklearn.metrics import confusion_matrix,recall_score,precision_score,f1_score

#フィルタ作成
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)
     
def weight_variable_he(shape,nodes):
    initial = tf.truncated_normal(shape, stddev=np.sqrt(2.0/nodes))
    return tf.Variable(initial)
     
def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x,W):
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding='VALID')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize = [1,1,2,1], strides=[1,1,2,1],padding = 'VALID')

def batch_normalization(shape, input):
    eps = 1e-5
    gamma = weight_variable([shape])
    beta = weight_variable([shape])
    mean,variance = tf.nn.moments(input,[0])
    return gamma * (input - mean) / tf.sqrt(variance+eps) + beta

def Activation(x):
    datas = tf.nn.relu(x)
    #datas = tf.nn.tanh(x)
    return datas



#define
#Data_Classes = 8#学習クラス数
steps = 3000 #エポック数　学習の回数
BATCH_SIZE = 600 #ミニバッチの一回の学習で使う量
#TBATCH_SIZE = 10
Validation = 0.8  #  ここの値で分割 ex. Validation = 0.4 →　0.4:0.6　に分割
CaptureNumber = 200 #データ数　ミニバッチの大きさ、初期は２００個のデータ
drop = 1.0 #dropoutの係数、NNのノードがこれをかけた数になる

allCSize = 4608 #全結合サイズ
filterSize = 64 #畳みこみフィルタ（カーネル）数 今回は 1 x filiterSize の大きさ

# 訓練データのフォルダのパス
train_folder_name = "train_folder_name"

# 検証データのフォルダのパス
test_folder_name = "test_folder_name"

#全訓練データとそのラベル
All_Datas = []
All_Label = []

#外部検証データとそのラベル
Ex_TestDatas=[]
Ex_Label = []

#並び順をシャッフルしたあとの訓練データ
All_SDatas =[]
All_SLabel = []

#分割後の訓練データ，学習するほう
train_data = []
train_label = []

#分割後の訓練データ，モデル評価するほう
test_data = []
test_label = []

#フォルダ中身のファイル名を取得
trainFolder = os.listdir(train_folder_name)
testFolder = os.listdir(test_folder_name)




#学習クラス数
Data_Classes = len(trainFolder)

# .DS_Storeがあるか検知
for i,d in enumerate(trainFolder):
    if d == ".DS_Store":
        Data_Classes = Data_Classes - 1
        print("DS!!!!!!!!!!!!!!!!!!!!!!!!!!!!")


print(Data_Classes)
print(len(testFolder))
print(train_folder_name)
print(test_folder_name)

#フォルダごとにみる
#訓練データの読み込み

dsflag = 0

for i,d in enumerate(trainFolder):
    #one_hot_vector生成用
    tmp = np.zeros(Data_Classes)
    if d != ".DS_Store":
        #フォルダ内のファイルのリストを取得
        files = os.listdir(train_folder_name + '/'+d)
        #print(files)

        #one_hot_vectorを作成
        tmp[i-dsflag] = 1
        #ファイル毎にみる
        for f in files:
                            
            #.DS_Storeをどかす
            if f != ".DS_Store":
                #どのファイルを見ているか確認用
                #print("load:"+f)

                datafile_path = train_folder_name + '/' + d+'/'+f

                #csvから読み込み
                data = np.loadtxt(datafile_path, delimiter=",")     #pbldata用

                #3x200のデータに整形
                x_csv = data[1, :]
                y_csv = data[2, :]
                z_csv = data[3, :]
                
#                 start = random.randint(0, len(x_csv) - CaptureNumber)
#                 end = start + CaptureNumber

#                 x_data = x_csv[start:end]
#                 y_data = y_csv[start:end]
#                 z_data = z_csv[start:end]


#                 flatdata = []
#                 flatdata.append(x_data)
#                 flatdata.append(y_data)
#                 flatdata.append(z_data)
                
                flatdata = []
                flatdata.append(x_csv)
                flatdata.append(y_csv)
                flatdata.append(z_csv)
          
                All_Datas.append(flatdata)
                #one_hot_vectorをラベルに追加
                All_Label.append(tmp)
            else:
                print(".ds_storeを除去")
    else:
        print(".ds_storeを抹殺")
        print(i)
        dsflag = dsflag + 1


dsflag = 0

#検証データの読み込み
for i, d in enumerate(testFolder):
    #one_hot_vector生成用
    tmp = np.zeros(Data_Classes)
    if d != ".DS_Store":
        # フォルダ内のファイルのリストを取得
        files = os.listdir(test_folder_name + '/' + d)
        #print(files)

        #one_hot_vectorを作成
        tmp[i-dsflag] = 1
        #ファイル毎にみる
        for f in files:
                            
            #.DS_Storeをどかす
            if f != ".DS_Store":
                #どのファイルを見ているか確認用
                #print("load:"+f)

                datafile_path = test_folder_name + '/' + d+'/'+f

                #csvから読み込み
                data = np.loadtxt(datafile_path, delimiter=",")     #pbldata用

                 #3x200のデータに整形
                x_csv = data[1, :]
                y_csv = data[2, :]
                z_csv = data[3, :]
                
                start = random.randint(0, len(x_csv) - CaptureNumber)
                end = start + CaptureNumber

                x_data = x_csv[start:end]
                y_data = y_csv[start:end]
                z_data = z_csv[start:end]


                flatdata = []
                flatdata.append(x_data)
                flatdata.append(y_data)
                flatdata.append(z_data)

                
                Ex_TestDatas.append(flatdata)
                #one_hot_vectorをラベルに追加
                Ex_Label.append(tmp)
            else:
                print(".ds_storeを除去")
    else:
        print("ex.ds_storeを抹殺")
        dsflag = dsflag + 1
  
print("Ex_Label")
print(Ex_Label[0])

#訓練データ順列のシャッフル
np.random.seed(seed=32)
Rindex = np.random.permutation(list(range(len(All_Datas))))
print(Rindex)
for k in Rindex:
    All_SDatas.append(All_Datas[k])
    All_SLabel.append(All_Label[k])

    
#訓練データを学習に使うやつとモデルの評価に使うやつの２種に分ける
numberV = int(len(All_SDatas)*(Validation)) #訓練データ数
testV = len(All_SDatas) - numberV  #テストデータ数
train_data = All_SDatas[:numberV] #0.９まで
test_data = All_SDatas[numberV+1:]
train_label = All_SLabel[:numberV]
test_label = All_SLabel[numberV+1:]

print(len(train_data))
print(len(test_data))

print(train_data[0])

#----------
#numpy行列へ
#----------

train_data = np.array(train_data)
train_label = np.array(train_label)

test_data = np.array(test_data)
test_label = np.array(test_label)

sess = tf.InteractiveSession()
x = tf.placeholder('float', shape=[None,3,CaptureNumber]) #3x200のデータ
y_ = tf.placeholder('float', shape=[None, Data_Classes]) #正解ラベル
keep_prob = tf.placeholder('float')
                                     
# -----------------------------------------
# 畳み込みニューラルネットワーク
#
# (convconv -> bn -> pool -> drop) x3，
# (fc->drop) x2
# -----------------------------------------

#第1ブロック
W_conv1_1 = weight_variable([1,5,3,filterSize]) #1*5のフィルタ　入力３ｃｈ　出力６４枚
b_conv1_1 = bias_variable([filterSize]) #出力６４ch
x_image = tf.reshape(x,[-1,1,CaptureNumber,3]) #-１はreshapeに適切な数N　１*２００のデータ　３ｃｈ
h_conv1_1 = conv2d(x_image, W_conv1_1)#+b_conv1_1 
tan1_1 = Activation(h_conv1_1)
bn1_1 = batch_normalization(filterSize,tan1_1)

W_conv1_2 = weight_variable([1,5,filterSize,filterSize])
b_conv1_2 = bias_variable([filterSize])
h_conv1_2 = conv2d(bn1_1, W_conv1_2)#+b_conv1_2
tan1_2 = Activation(h_conv1_2)
bn1_2 = batch_normalization(filterSize,tan1_2)

h_pool1 = max_pool_2x2(bn1_2)
h_pool1_drop = tf.nn.dropout(h_pool1, keep_prob)


# #第2ブロック
W_conv2_1 = weight_variable([1,5,filterSize,filterSize*2])
b_conv2_1 = bias_variable([filterSize*2])
h_conv2_1 = conv2d(h_pool1_drop, W_conv2_1)#+b_conv2_1
tan2_1 = Activation(h_conv2_1)
bn2_1 = batch_normalization(filterSize*2,tan2_1)

W_conv2_2 = weight_variable([1,5,filterSize*2,filterSize*2])
b_conv2_2 = bias_variable([filterSize*2])
h_conv2_2 = conv2d(bn2_1, W_conv2_2)#+b_conv2_2
tan2_2 = Activation(h_conv2_2)
bn2_2 = batch_normalization(filterSize*2,tan2_2)

h_pool2 = max_pool_2x2(bn2_2)
h_pool2_drop = tf.nn.dropout(h_pool2, keep_prob)

#

# #第3ブロック
W_conv3_1 = weight_variable([1,5,filterSize*2,filterSize*4])
b_conv3_1 = bias_variable([filterSize*4])
h_conv3_1 = conv2d(h_pool2_drop, W_conv3_1)#+b_conv3_1
tan3_1 = Activation(h_conv3_1)
bn3_1 = batch_normalization(filterSize*4,tan3_1)

W_conv3_2 = weight_variable([1,5,filterSize*4,filterSize*4])
b_conv3_2 = bias_variable([filterSize*4])
h_conv3_2 = conv2d(bn3_1, W_conv3_2)#+b_conv3_2
tan3_2 = Activation(h_conv3_2)
bn3_2 = batch_normalization(filterSize*4,tan3_2)


h_pool3 = max_pool_2x2(bn3_2)
h_pool3_drop = tf.nn.dropout(h_pool3, keep_prob)
h_pool3_flat = tf.reshape(h_pool3_drop, [-1,allCSize])

#全結合層
W_fc1 = weight_variable([allCSize, allCSize])
b_fc1 = bias_variable([allCSize])
h_fc1 = tf.nn.relu(tf.matmul(h_pool3_flat, W_fc1) + b_fc1)
bn4 = batch_normalization(allCSize,h_fc1)

# #dropout
h_fc1_drop = tf.nn.dropout(bn4, 1.0)

# #全結合層
W_fc2 = weight_variable([allCSize, allCSize])
b_fc2 = bias_variable([allCSize])

h_fc2= tf.nn.relu(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)
bn5 = batch_normalization(allCSize,h_fc1)

# #dropout
h_fc2_drop = tf.nn.dropout(bn5, 0.5)

#softmaxで出力#
W_fc4 = weight_variable([allCSize, Data_Classes])
b_fc4 = bias_variable([Data_Classes])
y_conv=tf.nn.softmax(tf.matmul(h_fc2_drop, W_fc4) + b_fc4)




#------------------
# 評価と損失関数
#------------------

#L2ノルムを足そうとした残骸
# L2_sqr = tf.nn.l2_loss(W_conv1) + tf.nn.l2_loss(W_conv2)+tf.nn.l2_loss(W_conv3) + tf.nn.l2_loss(W_fc1) + tf.nn.l2_loss(W_fc2)
# lambda_2 = 0.01

#クロスエントロピー誤差関数，1e-7を足して，勾配消失を防止
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_*tf.log(y_conv+1e-7),reduction_indices=[1]))
#loss = cross_entropy + lambda_2*L2_sqr
# cross_entropy = -tf.reduce_sum(y_*tf.log(tf.clip_by_value(y_conv,1e-10,1.0)))

#重みの最適化
train_step = tf.train.AdamOptimizer(1e-3).minimize(cross_entropy)
#　推定結果のy_convと正解ラベルy_が同じかどうか判定，correct_predictionがbool配列になってる
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_,1))
# 正答率
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
# run
sess.run(tf.global_variables_initializer())



#
#学習器の保存処理
saver = tf.train.Saver()
ckpt = tf.train.get_checkpoint_state('./')
cwd = os.getcwd()


#処理時間計測
startTime = datetime.datetime.today()

#ログ
learn = open('log.csv','w')
result = open('result.txt','w')


learn.write("class,step\n" )
learn.write("%g,%g\n"%(Data_Classes,steps))
learn.write("step,train_a,test_a\n" )

#学習ループ
#配列に入れた加速度データから，ランダムにデータを選択，さらにそこから3x200に切り出して学習にかける
for i in range(steps):

    #加速度データの中からランダムにデータを選択
    batch_mask = np.random.choice(len(train_data),BATCH_SIZE)
    tbatch_mask = np.random.choice(len(test_data),BATCH_SIZE)

    #バッチ配列
    train_data_batch = []
    train_label_batch = train_label[batch_mask]
    test_data_batch = []

    #3x200に切り出し_訓練データ
    for g in batch_mask:
#         rdata = train_data[g]
#         train_data_batch.append(rdata)

        b_data = train_data[g]

        b_x = b_data[0]
        b_y = b_data[1]
        b_z = b_data[2]
        
        rdata = []
        start = random.randint(0, len(b_x) - CaptureNumber)
        end = start + CaptureNumber

        x_data = b_x[start:end]
        y_data = b_y[start:end]
        z_data = b_z[start:end]

        rdata.append(x_data)
        rdata.append(y_data)
        rdata.append(z_data)

        train_data_batch.append(rdata)

    # 3x200に切り出し_テストデータ
    for g in range(len(test_data)):
#         rdata = test_data[g]
#         test_data_batch.append(rdata)
        b_data = test_data[g]

        b_x = b_data[0]
        b_y = b_data[1]
        b_z = b_data[2]

        rdata = []
        start = random.randint(0, len(b_x) - CaptureNumber)
        end = start + CaptureNumber

        x_data = b_x[start:end]
        y_data = b_y[start:end]
        z_data = b_z[start:end]

        rdata.append(x_data)
        rdata.append(y_data)
        rdata.append(z_data)

        test_data_batch.append(rdata)

    # 学習の実行　x＝データy＿＝ラベル keep_prob=dropoutでどれだけノード消すか
    w_out, _ = sess.run([y_conv, train_step],feed_dict={x: train_data_batch, y_: train_label_batch, keep_prob:drop})

    #精度の計算
    train_accuracy = accuracy.eval(feed_dict={
        x: train_data_batch, y_: train_label_batch, keep_prob:drop})

    # 10ステップごとに精度を記録
    if i % 10 == 0 :
        test_accuracy = accuracy.eval(feed_dict={
            x: test_data_batch, y_: test_label, keep_prob: drop})

        print("step %d, training accuracy %g   test accuracy %g" % (i, train_accuracy, test_accuracy))
        learn.write("%d,%g,%g\n" % (i, train_accuracy, test_accuracy))

    #終了一回前にいろいろ作成
    if i == steps-1:

        # confusion matrix 作成
        y_p = tf.argmax(y_conv, 1)
        val_accuracy, y_pred = sess.run([accuracy, y_p], feed_dict={x: test_data_batch, y_: test_label, keep_prob: drop})
        y_true = np.argmax(test_label, 1)
        con = confusion_matrix(y_true, y_pred)
        print(con)

        #検証データ分類用
        #y_pred_1 = sess.run([y_p],feed_dict={x: Ex_TestDatas, keep_prob: drop})
        # y_pred2 = y_pred_1[0]
        # print(y_pred2)
        # print(len(y_pred2))



        print("ccn test 正答率 %g" % accuracy.eval(feed_dict={
            x: test_data_batch, y_: test_label, keep_prob: drop
        }))

#         print("ccn realtime test 正答率 %g" % accuracy.eval(feed_dict={
#             x: Ex_TestDatas, y_: Ex_Label, keep_prob: drop
#         }))

        #処理時間計測，終わり
        endTime = datetime.datetime.today()

        print("")
        print("開始時刻 %s" % (startTime))
        print("終了時刻 %s" % (endTime))

        learn.write("\n開始時刻 %s\n" % (startTime))
        learn.write("終了時刻 %s\n" % (endTime))
        learn.write("ccn test 正答率 %g" % accuracy.eval(feed_dict={
            x: test_data_batch, y_: test_label, keep_prob: drop
        }))

        #confusion matrixについて，精度を百分率表記したcon1と何が何回正解したかを表すconの両方つくる
        con_sum = np.sum(con, axis=1)
        con1 = []
        for a in range(len(con)):
            c = con[a] / con_sum[a]
            con1.append(c)

        print(con_sum)
        print(con)
        result.write("Confusion Matrix :\n")
        # np.savetxt('result.txt',con,fmt="%0.2f")
        np.savetxt('confusion-no.csv', con, fmt="%0.3f", delimiter=',')
        np.savetxt('confusion.csv', con1, fmt="%0.3f", delimiter=',')


        ##学習器の保存処理２
saver.save(sess,cwd+"\\0model.ckpt")



learn.close()

DS!!!!!!!!!!!!!!!!!!!!!!!!!!!!
8
9
train_folder_name
test_folder_name
.ds_storeを抹殺
8
ex.ds_storeを抹殺
Ex_Label
[1. 0. 0. 0. 0. 0. 0. 0.]
[131 600 550 217  91 242 296 196 388  13 585 797 683  74  51 653 716 133
 779 383 749 587 315  60 380 497 460 313 151 220  81  92 726 455 293  41
 361 538 759 660 237 186  99 233 617 208 646 381 222 431  19 736 218 662
 545 693 224  97 373 443  39 549 637 789  17 415  49 272 757 471 370  40
 275 639 135 783 402 346 722 401 250 570 199  56 701 335 193  50 153 195
 144 121 295 673  11 141 760 689 657 413 277 559 725 498 476 365 485 528
 225  85  84 741  32 761 464  45 322 687 656 394  52 446 343 219 404  43
 113 546 174 565 245  86 630 171 557 504 111 649 780 215 750 521  16 602
 558 492 289  96   8 642 730 796  68 316 495 532 169 360 205 426  82 615
 633  79 685 254 281  12 677 513 763 392 595 279  47 756 317 239 432 400
 419 794 672 799 238  22 137 702 369 774 522 436   1 334 427   0 283  37
 311 182  65 132 411 740 732 619  70 188  30 671 330 606 755 2

/usr/local/lib/python3.6/dist-packages/tensorflow/python/client/session.py:1702: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


step 0, training accuracy 0.373333   test accuracy 0.2
step 10, training accuracy 0.59   test accuracy 0.4
step 20, training accuracy 0.645   test accuracy 0.6125
step 30, training accuracy 0.683333   test accuracy 0.6625
step 40, training accuracy 0.703333   test accuracy 0.65
step 50, training accuracy 0.733333   test accuracy 0.7125
step 60, training accuracy 0.755   test accuracy 0.6875
step 70, training accuracy 0.783333   test accuracy 0.70625
step 80, training accuracy 0.771667   test accuracy 0.75625
step 90, training accuracy 0.806667   test accuracy 0.81875
step 100, training accuracy 0.85   test accuracy 0.76875
step 110, training accuracy 0.836667   test accuracy 0.78125
step 120, training accuracy 0.846667   test accuracy 0.70625
step 130, training accuracy 0.83   test accuracy 0.84375
step 140, training accuracy 0.87   test accuracy 0.84375
step 150, training accuracy 0.856667   test accuracy 0.83125
step 160, training accuracy 0.881667   test accuracy 0.83125
step 170, t